# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

gemini_api_key = os.getenv("GEMINI_API_KEY")
openai = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
MODEL = "gemini-2.5-flash-lite"

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [01:22<00:00, 16.47s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Harbor Freight Tools Member Deals: Up to 53% off + shipping from $6.99\nDetails: Harbor Freight Tools Inside Track Club Members get up to 50% off nearly 20 items. (Not a member? It's free to join.) You will find savings on batteries, tool kits, drill bits, sanders, saws, and much more. Shipping starts at $6.99, or many items are also available in store for purchase. Shop Now at Harbor Freight Tools\nFeatures: \nURL: https://www.dealnews.com/Harbor-Freight-Tools-Member-Deals-Up-to-53-off-shipping-from-6-99/21760433.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: EcoFlow Delta 2 1,024Wh Portable Power Station for $490 + free shipping
Details: It's $19 under Amazon and the lowest price we could find. Buy Now at Walmart
Features: AC charge input and output  solar input (solar panel not included)  4 USB charging ports 2 USB-C charging ports Model: EFD330-2
URL: https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-2-1-024-W

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='The EcoFlow Delta 2 is a portable power station with a 1,024Wh capacity. It offers AC charge input and output, and a solar input option (solar panel not included). For charging devices, it features 4 USB ports and 2 USB-C charging ports, providing versatile connectivity for various electronic devices. This model is the EcoFlow Delta 2, model EFD330-2.', price=490.0, url='https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-2-1-024-Wh-Portable-Power-Station/473744.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='This is a Samsung QN80F Series 100-inch 4K HDR 120Hz Neo QLED Smart TV from the 2025 lineup. It features a 3840x2160 native resolution, HDR10+ and HLG support, and a 120Hz refresh rate. The TV runs on the Tizen Smart OS and is compatible with Alexa and Google Assistant. It includes 4 HDMI inputs for connectivity. This purchase also includes a free 65-inch U8000F 4K Smart TV.', price=6000.0, url='https://www.dealnews.com/products/Samsung/Samsung-QN80-F-Series-QN100-QN80-FFXZA-100-4-K-HDR-120-Hz-Neo-QLED-Smart-TV-2025/492808.html?iref=rss-c142'), Deal(product_description="The EcoFlow Delta 2 is a portable power station with a 1,024Wh capacity. It features AC charge input and output, a solar input port (solar panel not included), four USB charging ports, and two USB-C charging ports. This unit is designed for versatile power needs, whether you're camping or dealing with power outages.", price=490.0, url='https://www.dealnews.com/products/Eco-